In [35]:
import h5py
from sklearn.model_selection import train_test_split
import numpy as np

In [28]:
data = h5py.File('data/Q1/MNIST_Subset.h5','r+')
X=data['X'][:]
Y=data['Y'][:]

In [29]:
X.shape,Y.shape

((14251, 28, 28), (14251,))

In [30]:
X = X.reshape(X.shape[0],-1)

In [31]:
X=X/255

In [32]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

In [33]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((11400, 784), (2851, 784), (11400,), (2851,))

In [39]:
np.unique(Y_train)

array([7, 9], dtype=uint8)

In [40]:
#as we only have 2 classes
Y_train = np.where(Y_train==7,0,1)

In [41]:
np.unique(Y_train)

array([0, 1])

In [34]:
#hyperparameters
epocs = 10
learning_rate = 0.01

In [36]:
def sigmoid(z,derative = False):
    if derative:
        return z*(1-z)
    return 1.0/(1.0+np.exp(-z))